# Criar grafo bipartido

In [1]:
import numpy as np
import pandas as pd
import ast
import networkx as nx

In [2]:
# declare a list tasks whose products you want to use as inputs
upstream = None
product = None
edge_percent = 0.1 # porcentagem de quantidade do peso da aresta para ela ser removida ou não 
df_input_path = '../../dados/X_trainToronto.csv'
df_review_input_path = '../../dados_produzidos/reviewsTrainTorontoSentiment.csv'

In [3]:
# Parameters
df_input_path = "../../dados/X_trainToronto.csv"
df_review_input_path = "../../dados_produzidos/reviewsTrainTorontoSentiment.csv"
edge_percent = 0.1
product = {
    "data": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/graph_feature.parquet",
    "graph": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/graph.gml",
    "nb": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/output_graph_feature.ipynb",
}


In [4]:
df_reg = pd.read_csv(df_input_path)
df_reg = df_reg[['business_id', 'review_count']]
df_reg

,business_id,review_count
0,vHzWmPWHN4J1hRR3W3AMQg,99
1,15to24Q-otAHmto7FzsWRg,3
2,8aqKdf4G4AAir8k_Kdslvg,18
3,uxU1vr5AhhkTQ83X0bpeyg,3
4,f702hTJoqdR34Jn23C7d1A,3
...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,16
17578,o2811xZ0ufRi7WcsqWiGzw,4
17579,v9T9m_TbpLuaC2Kud5UbLA,8
17580,bzGnAIe9OiBHGWJE-dZ_8g,9


## Fazer grafo bipartido de locais
Se uma pessoa fez uma avaliação de dois estabelecimentos, uma aresta será formada entre eles

In [5]:
df_review = pd.read_csv(df_review_input_path)[['business_id', 'user_id']]
df_review

,business_id,user_id
0,--DaPTJW3-tB1vP-PfdTEg,Y2TBSd3ExydbgEnVxAt_QA
1,--DaPTJW3-tB1vP-PfdTEg,Rp-cSUHm-EKBxgBS73PNNg
2,--DaPTJW3-tB1vP-PfdTEg,2fXZuNKP7Bo-yRrINVbVVA
3,--DaPTJW3-tB1vP-PfdTEg,fn3Wim-2j042IEDe6VhJFQ
4,--DaPTJW3-tB1vP-PfdTEg,CD9dSllum_L_OvpRW0YH0w
...,...,...
490958,zzvlwkcNR1CCqOPXwuvz2A,YyV_UBEAoTMgwImiKnWYTQ
490959,zzvlwkcNR1CCqOPXwuvz2A,pkRISP8QgKMDPQlr8eNrdA
490960,zzvlwkcNR1CCqOPXwuvz2A,yDRw0UY7FhjDAs-q1bvRhg
490961,zzvlwkcNR1CCqOPXwuvz2A,U708fyOqNaBU0IQoE6E7WQ


In [6]:
df_index_to_business = pd.DataFrame(df_reg['business_id'])
df_business_to_index = df_reg['business_id'].reset_index().set_index('business_id')
df_index_to_review_count = df_reg['review_count'].values

In [7]:
import itertools
from tqdm import tqdm

tqdm.pandas()
G = nx.Graph()
G.add_nodes_from(df_index_to_business.index)
    
def add_edges(rows):
    if len(rows) < 2:
        return
    business_ids = [df_business_to_index.loc[x].iloc[0] for x in rows['business_id'].unique()]
    for comb in itertools.combinations(business_ids, 2):
        if comb in G.edges:
            G.edges[comb[0], comb[1]]['weight'] += 1
        else:
            G.add_edge(comb[0], comb[1], weight=1)

df_review.groupby('user_id').progress_apply(add_edges)
G.number_of_edges()

  0%|                                                                                                                                         | 0/109250 [00:00<?, ?it/s]

  0%|                                                                                                                                | 11/109250 [00:00<43:53, 41.48it/s]

  0%|▍                                                                                                                            | 360/109250 [00:00<01:27, 1246.93it/s]

  1%|▊                                                                                                                            | 747/109250 [00:00<00:53, 2044.45it/s]

  1%|█▏                                                                                                                          | 1074/109250 [00:00<00:44, 2428.22it/s]

...

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏| 108559/109250 [00:51<00:00, 2291.84it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 108863/109250 [00:51<00:00, 2474.17it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 109141/109250 [00:51<00:00, 2553.13it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 109250/109250 [00:51<00:00, 2115.04it/s]

8442907

In [8]:
edges_to_remove = []
for u,v,a in tqdm(G.edges(data=True), total=G.number_of_edges()):
    weight = a['weight']
    u_review_count = df_index_to_review_count[u]
    v_review_count = df_index_to_review_count[v]
    if float(weight) < edge_percent * float(min(u_review_count, v_review_count)):
        edges_to_remove.append((u, v))

  0%|                                                                                                                                        | 0/8442907 [00:00<?, ?it/s]

  1%|▉                                                                                                                       | 69144/8442907 [00:00<00:12, 691417.58it/s]

  2%|██                                                                                                                     | 143917/8442907 [00:00<00:11, 724526.24it/s]

  3%|███                                                                                                                    | 218873/8442907 [00:00<00:11, 735950.07it/s]

  3%|████                                                                                                                   | 292469/8442907 [00:00<00:11, 701284.60it/s]

...

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏ | 8316896/8442907 [00:13<00:00, 373938.82it/s]

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊ | 8354465/8442907 [00:13<00:00, 352969.87it/s]

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 8389982/8442907 [00:13<00:00, 317227.31it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 8422316/8442907 [00:13<00:00, 282481.24it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8442907/8442907 [00:13<00:00, 613511.78it/s]

In [9]:
for u,v in tqdm(edges_to_remove):
    G.remove_edge(u, v)

  0%|                                                                                                                                        | 0/5233404 [00:00<?, ?it/s]

  1%|█▍                                                                                                                      | 62530/5233404 [00:00<00:08, 625276.74it/s]

  2%|██▉                                                                                                                    | 126963/5233404 [00:00<00:08, 636472.23it/s]

  4%|████▍                                                                                                                  | 192918/5233404 [00:00<00:07, 646998.84it/s]

  5%|█████▊                                                                                                                 | 257907/5233404 [00:00<00:07, 648131.92it/s]

...

 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌     | 4991418/5233404 [00:07<00:00, 705761.66it/s]

 97%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▏   | 5062032/5233404 [00:07<00:00, 689796.46it/s]

 98%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋  | 5132078/5233404 [00:07<00:00, 692924.23it/s]

 99%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 5201446/5233404 [00:07<00:00, 681860.50it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5233404/5233404 [00:07<00:00, 669943.79it/s]

In [10]:
page_rank = nx.pagerank(G, weight='weight')
df_reg['graph_page_rank'] = pd.Series(page_rank)

In [11]:
closeness = nx.closeness_centrality(G)
df_reg['graph_closeness_centrality'] = pd.Series(closeness)

In [12]:
betweeness = nx.betweenness_centrality(G, weight='weight', k=1000)
df_reg['graph_betweenness_centrality'] = pd.Series(betweeness)

In [13]:
nx.write_gml(G, product['graph'])

In [14]:
df_reg.to_parquet(product['data'])